# Cloud Firestore

[Cloud Firestore](https://cloud.google.com/firestore) is a serverless document-oriented database that scales to meet any demand. 

Load and store documents from `Firestore` Native.

## Pre-reqs

In [ ]:
%pip install langchain-google-firestore

In [3]:
from langchain_google_firestore import FirestoreLoader, FirestoreSaver

## Basic Usage

### Load from Collection or SubCollection

In [6]:
loader_collection = FirestoreLoader("Collection")
loader_subcollection = FirestoreLoader("Collection/doc/SubCollection")


data_collection = loader_collection.load()
data_subcollection = loader_subcollection.load()

### Load a single Document

In [ ]:
from google.cloud import firestore

client = firestore.Client()
doc_ref = client.collection("foo").document("bar")

loader_document = FirestoreLoader(doc_ref)

data = loader_document.load()

### Load from CollectionGroup or Query

In [ ]:
from google.cloud.firestore import CollectionGroup, Query, FieldFilter

col_ref = client.collection("col_group")
collection_group = CollectionGroup(col_ref)

loader_group = FirestoreLoader(collection_group)

col_ref = client.collection("collection")
query = col_ref.where(filter=FieldFilter("region","==","west_coast"))

loader_query = FirestoreLoader(query)

## Customize Document Page Content & Metadata

The arguments of `page_content_fields` and `metadata_fields` will specify the Firestore Document fields to be written into LangChain Document `page_content` and `metadata`.

In [8]:
loader = FirestoreLoader(
    source="foo/bar/subcol",
    page_content_fields=['data_field'],
    metadata_fields=['metadata_field']
)

data = loader.load()

### Customize Page Content Format

When the `page_content` contains only one field the information will be the field value only. Otherwise the `page_content` will be in JSON format.

## Save Documents

`FirestoreSaver` can store Documents into Firestore. By default it will try to extract the Document reference from the metadata

In [ ]:
saver = FirestoreSaver()

saver.upsert_documents(data)

### Save Documents without reference

If a collection is specified the documents will be stored with an auto generated id

In [ ]:
saver = FirestoreSaver("Collection")

saver.upsert_documents(data)

### Save Documents with other references

In [ ]:
doc_ids = ["AnotherCollection/doc_id", "foo/bar"]
saver = FirestoreSaver()

saver.upsert_documents(documents=data, document_ids=doc_ids)

### Delete Documents

In [ ]:
saver = FirestoreSaver()

saver.delete_documents(data)

# The Documents will be ignored and only the document ids will be used.
saver.delete_documents(data, doc_ids)

## Customize Connection & Authentication

In [ ]:
from google.auth import compute_engine
from google.cloud.firestore import Client

client = Client(
    database = "non-default-db",
    creds=compute_engine.Credentials())
loader = FirestoreLoader(
    source="foo"
    client=client,
)